In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report

In [2]:
# 2.1.1 Merge datasets
historical_data = pd.read_csv('Historical-transaction-data.csv')
store_data = pd.read_csv('Store-info.csv')
merged_data = historical_data.merge(store_data, on='shop_id', how='left')

In [3]:
# 2.1.2 Handle missing values
training_data = merged_data[~merged_data['shop_profile'].isnull()]
prediction_data = merged_data[merged_data['shop_profile'].isnull()]

In [4]:
# 2.1.3 Feature engineering
training_data['total_sales'] = training_data['quantity_sold'] * training_data['item_price']
training_data['transaction_month'] = pd.to_datetime(training_data['transaction_date']).dt.month

C:\Users\naham\AppData\Local\Temp\ipykernel_21976\1301188028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['total_sales'] = training_data['quantity_sold'] * training_data['item_price']
C:\Users\naham\AppData\Local\Temp\ipykernel_21976\1301188028.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['transaction_month'] = pd.to_datetime(training_data['transaction_date']).dt.month


In [5]:
# 2.1.3 Feature engineering
# Create new features using .loc[] accessor to avoid SettingWithCopyWarning
training_data.loc[:, 'total_sales'] = training_data.loc[:, 'quantity_sold'] * training_data.loc[:, 'item_price']
training_data.loc[:, 'transaction_month'] = pd.to_datetime(training_data.loc[:, 'transaction_date']).dt.month

# 2.4 Predicting Missing Store Profiles
# Feature engineering for the prediction dataset using .loc[] accessor
prediction_data.loc[:, 'total_sales'] = prediction_data.loc[:, 'quantity_sold'] * prediction_data.loc[:, 'item_price']
prediction_data.loc[:, 'transaction_month'] = pd.to_datetime(prediction_data.loc[:, 'transaction_date']).dt.month


c:\Users\naham\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\Users\naham\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [6]:
# Create a separate copy of the DataFrame slice and perform feature engineering
training_data = training_data.copy()
training_data['total_sales'] = training_data['quantity_sold'] * training_data['item_price']
training_data['transaction_month'] = pd.to_datetime(training_data['transaction_date']).dt.month

# Feature engineering for the prediction dataset
prediction_data = prediction_data.copy()
prediction_data['total_sales'] = prediction_data['quantity_sold'] * prediction_data['item_price']
prediction_data['transaction_month'] = pd.to_datetime(prediction_data['transaction_date']).dt.month


In [7]:
training_data.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile,total_sales,transaction_month
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2,678,Moderate,440,12
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,668,Moderate,440,10
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2,678,Moderate,320,12
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2,678,Moderate,300,12
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5,668,Moderate,1050,10


In [8]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473974 entries, 0 to 473973
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   item_description  438046 non-null  object 
 1   transaction_date  473974 non-null  object 
 2   invoice_id        467654 non-null  float64
 3   customer_id       473974 non-null  object 
 4   shop_id           473974 non-null  object 
 5   item_price        473974 non-null  int64  
 6   quantity_sold     473974 non-null  int64  
 7   shop_area_sq_ft   473974 non-null  int64  
 8   shop_profile      387341 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 36.2+ MB


In [9]:
import pandas as pd

# Assuming 'merged_data' is your merged dataset
# Create a copy of the merged data
merged_data_copy = merged_data.copy()

# Feature engineering: Calculate total sales
merged_data_copy['total_sales'] = merged_data_copy['quantity_sold'] * merged_data_copy['item_price']

# Aggregate data by shop_id
aggregated_data = merged_data_copy.groupby('shop_id').agg({
    'total_sales': ['sum', 'mean', 'max', 'min'],
    'item_price': ['sum', 'mean', 'max', 'min'],
    'quantity_sold': ['sum', 'mean', 'max', 'min'],
    'transaction_date': ['count'],
    'shop_area_sq_ft': 'first',
    'shop_profile': 'first'
}).reset_index()

# Flatten the column names after aggregation
aggregated_data.columns = ['_'.join(col).strip() for col in aggregated_data.columns.values]

# Rename the columns if necessary
aggregated_data.rename(columns={
    'shop_id_': 'shop_id',
    'transaction_date_count': 'transaction_count',
    'shop_area_sq_ft_first': 'shop_area_sq_ft',
    'shop_profile_first': 'shop_profile'
}, inplace=True)

# Display the aggregated data
print(aggregated_data.head())



   shop_id  total_sales_sum  total_sales_mean  total_sales_max  \
0  SHOP001          1485285        735.289604           367920   
1  SHOP002          3084455        642.594792            82320   
2  SHOP003          1922995        525.552063            33660   
3  SHOP004          1571700        454.379879            14000   
4  SHOP005          2148520        511.430612           117390   

   total_sales_min  item_price_sum  item_price_mean  item_price_max  \
0                0          433145       214.428218            5040   
1                0         1090135       227.111458            2880   
2                0          769685       210.353922            1870   
3                0          711050       205.565192            1280   
4                0          842995       200.665318            2900   

   item_price_min  quantity_sold_sum  quantity_sold_mean  quantity_sold_max  \
0              35               3942            1.951485                 73   
1              35 